### 전체 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 청약매물 목록 정보
df_estate_list = pd.read_csv('../datasets/한국부동산원_청약홈_APT 분양정보_20250225.csv', encoding='cp949')

# 청약 신청자, 당첨자 정보
df_applicant_list = pd.read_csv('../datasets/한국부동산원_지역별 청약 신청자 정보_20250131.csv', encoding='cp949')
df_winner_list = pd.read_csv('../datasets/한국부동산원_지역별 청약 당첨자 정보_20250131.csv', encoding='cp949')

# 청약 당첨가점, 경쟁률 정보
df_estate_winner_score_list = pd.read_csv('../datasets/한국부동산원_청약홈_APT 가점정보_20250225.csv', encoding='cp949')
df_estate_competitition_rate_list = pd.read_csv('../datasets/한국부동산원_청약홈_APT 경쟁률_20250225.csv', encoding='cp949')

### 병합 1) 청약 매물 목록 정보 + 신청자, 당첨자 정보

In [ ]:
# 청약매물 목록 정보에서 청약접수시작월, 당첨자발표월 컬럼 추가

df_estate_list['청약접수시작월'] = pd.to_datetime(df_estate_list['청약접수시작일'], format='%Y-%m-%d').dt.strftime('%Y-%m')
df_estate_list['당첨자발표월'] = pd.to_datetime(df_estate_list['당첨자발표일'], format='%Y-%m-%d').dt.strftime('%Y-%m')
df_estate_list[['공급지역명', '청약접수시작월', '당첨자발표월']].head()

In [ ]:
# 청약 신청자 정보 확인

df_applicant_list.head()

In [ ]:
# 청약 신청자 정보 칼럼 가공

df_applicant_list['연월'] = pd.to_datetime(df_applicant_list['연월']).dt.strftime('%Y-%m')

# 신청자 접두사 추가
df_applicant_list = df_applicant_list.rename(columns={
    '30대 이하': '신청자 30대 이하',
    '40대': '신청자 40대',
    '50대': '신청자 50대',
    '60대 이상': '신청자 60대 이상'
})

df_applicant_list.head()

In [ ]:
# 청약 목록 데이터프레임에 연월과 시도에 따라 신청자 데이터 병합

df_estate_list = pd.merge(
    df_estate_list, 
    df_applicant_list,
    left_on=['공급지역명', '청약접수시작월'], 
    right_on=['시도', '연월'],
    how='inner',
)

df_estate_list.drop(columns=['청약접수시작월','연월', '시도'], inplace=True)

df_estate_list.head()

In [ ]:
# 청약 당첨자 정보 확인

df_winner_list.head()

In [ ]:
# 청약 당첨자 정보 칼럼 가공

df_winner_list['연월'] = pd.to_datetime(df_winner_list['연월']).dt.strftime('%Y-%m')

#  당첨자 접두사 추가
df_winner_list = df_winner_list.rename(columns={
    '30대 이하': '당첨자 30대 이하',
    '40대': '당첨자 40대',
    '50대': '당첨자 50대',
    '60대 이상': '당첨자 60대 이상'
})

df_winner_list.head()

In [ ]:
# 청약 목록 데이터프레임에 연월과 시도에 따라 당첨자 데이터 병합

df_estate_list = pd.merge(
    df_estate_list, 
    df_winner_list,
    left_on=['공급지역명', '당첨자발표월'], 
    right_on=['시도', '연월'],
    how='inner',
)

df_estate_list.drop(columns=['당첨자발표월','연월', '시도'], inplace=True)

df_estate_list.head()

### 병합 2) 청약 매물 목록 정보 + 당첨가점, 경쟁률 정보

In [ ]:
# 당첨가점 데이터 확인

df_estate_winner_score_list

In [ ]:
# 경쟁률 데이터 확인

df_estate_competitition_rate_list

In [ ]:
# 경쟁률 데이터에 당첨가점 데이터 병합

def apply_score(row, score_column):
    rank = row['순위']

    # 2순위 인 경우 nan값으로 채운다
    if rank == 2:
        return np.nan

    # 조건에 맞는 데이터 필터링
    matched_rows = df_estate_winner_score_list[
        (df_estate_winner_score_list['공고번호'] == row['공고번호']) &
        (df_estate_winner_score_list['모델번호'] == row['모델번호']) &
        (df_estate_winner_score_list['주택형'] == row['주택형'])
    ]

    # 조건에 맞는 행이 없을 경우 nan값으로 채운다
    if matched_rows.empty:
        return np.nan 

    # 존재하면 첫번째 행의 값 리턴
    return matched_rows.iloc[0][score_column]

score_column_list = ['최저당첨가점', '평균당첨가점', '최고당첨가점']

for score_column in score_column_list:
    df_estate_competitition_rate_list[score_column] = df_estate_competitition_rate_list.apply(apply_score, args=(score_column,),  axis=1)

df_estate_competitition_rate_list

In [ ]:
# 청약 매물 목록 데이터에 청약 매물 항목별 경쟁률, 당첨가점 병합

df_estate_list= pd.merge(
    df_estate_list, 
    df_estate_competitition_rate_list, 
    on=['주택관리번호', '공고번호'], 
    how='inner'
).reset_index(drop=True)

df_estate_list

### 3) 병합 데이터 저장

In [ ]:
df_estate_list.to_csv('병합_청약매물_목록_정보.csv', index=False, encoding='cp949')

In [ ]:
df_estate_list_merged = pd.read_csv('병합_청약매물_목록_정보.csv', encoding='cp949')
df_estate_list_merged = df_estate_list_merged[df_estate_list_merged['공급지역명'].isin(['서울', '경기', '인천'])].reset_index(drop=True)
df_estate_list_merged

### 4) 공급금액 크롤링 후 추가

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# 청약 매물 공급금액 정보 가져오기

def get_estate_price(estate_id):
    query = f'houseManageNo={estate_id}&pblancNo={estate_id}'
    URL = f'https://www.applyhome.co.kr/ai/aia/selectAPTLttotPblancDetail.do?{query}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(URL, headers=headers)
    response.content

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, 'lxml')

    # 공급금액 존재하는 테이블 찾기
    table = soup.findAll('table')[-2]

    # DataFrame 생성
    df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0] # converters: 각 컬럼의 데이터를 문자열로 변환

    return df

df_estate_price = get_estate_price(2020000009)
df_estate_price

In [ ]:
# 청약 매물 목록 데이터에 공급금액(최고가 기준) 컬럼 추가

# 청약 매물 목록 id 가져오기
estate_ids = df_estate_list_merged['공고번호'].unique().tolist()
estate_ids

# 빈 문자열 존재하여 제거
df_estate_list_merged['주택형'] = df_estate_list_merged['주택형'].str.strip()

for estate_id in estate_ids:
    try:
        # 청약 매물 공급금액 정보 가져오기
        df_estate_price = get_estate_price(estate_id)

        # 청약 매물 목록 데이터에 공급금액(최고가 기준) 컬럼 추가
        for index in range(len(df_estate_price)):
            estate_type = str(df_estate_price.loc[index, '주택형']).strip()
            estate_price = float(df_estate_price.loc[index, '공급금액(최고가 기준)']) * 10000

            mask = (df_estate_list_merged['공고번호'] == estate_id) & (df_estate_list_merged['주택형'] == estate_type)
            df_estate_list_merged.loc[mask, '공급금액(최고가 기준)'] = estate_price
    except:
        print(f'{estate_id} 데이터를 가져오는데 실패했습니다.')

df_estate_list_merged

In [ ]:
df_estate_list_merged['공급금액(최고가 기준)'].isna().sum()

In [ ]:
df_estate_list_merged.columns

In [ ]:
df_estate_list_merged.to_csv('청약매물_공급금액 (서울, 경기, 인천).csv', index=False, encoding='cp949')